In [1]:
# Dependencies
from census import Census
from config import (api_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from uszipcode import SearchEngine, SimpleZipcode, Zipcode


# Census API Key
c = Census(api_key, year=2017)

In [2]:
restdata_df = pd.read_csv("../Project1_GroupSix-/county_zip_data.csv")
restdata_df.head()

,Zip_Codes,Lat,Lon
0,75001,32.96,-96.84
1,75002,33.08,-96.61
2,75006,32.95,-96.89
3,75007,33.00,-96.90
4,75009,33.30,-96.80


In [3]:
restdata_df.columns

Index(['Zip_Codes', 'Lat', 'Lon'], dtype='object')

In [4]:
restdata_df.columns = ['Zip_Codes', 'Lat', 'Lng']

In [5]:
restdata_df.head()

,Zip_Codes,Lat,Lng
0,75001,32.96,-96.84
1,75002,33.08,-96.61
2,75006,32.95,-96.89
3,75007,33.00,-96.90
4,75009,33.30,-96.80


In [6]:
# set up additional columns to hold information
restdata_df['Business'] = ""
restdata_df['Cuisine'] = ""
restdata_df['Price_Level'] = ""
restdata_df['Rating'] = ""

restdata_df.head()

,Zip_Codes,Lat,Lng,Business,Cuisine,Price_Level,Rating
0,75001,32.96,-96.84,,,,
1,75002,33.08,-96.61,,,,
2,75006,32.95,-96.89,,,,
3,75007,33.00,-96.90,,,,
4,75009,33.30,-96.80,,,,


# Call data from API ATTEMPT ONE 

In [9]:
# params dictionary to update each iteration
params = {
    "radius": 50,
    "types": "restaurant",
    "keyword": "ethnicity",
    "key": gkey

# Use the lat/lng we recovered to identify airports
for index, row in restdata_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    rest_request = requests.get(base_url, params=params)
    
# print the rest_request url, avoid doing for public github repos in order to avoid exposing key
# print(rest_request.url)

    # convert to json
    rest_request = rest_request.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        restdata_df.loc[index, "Business"] = rest_request["results"][0]["name"]
        restdata_df.loc[index, "Cuisine"] = rest_request["results"][0]["ethnicity"]
        restdata_df.loc[index, "Rating"] = rest_request["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

SyntaxError: invalid syntax (<ipython-input-9-d54bd3d24dad>, line 9)

# API CALL APPTEMP TWO by cuisine type

In [14]:
coord_pair = [(xy) for xy in zip(restdata_df.Lat, restdata_df.Lng)]


In [13]:
restdata_df["Coordinates"]= coord_pair

locations = restdata_df["Coordinates"]
restdata_df.head()

,Zip_Codes,Lat,Lng,Business,Cuisine,Price_Level,Rating,Coordinates
0,75001,32.96,-96.84,,,,,"(32.96, -96.84)"
1,75002,33.08,-96.61,,,,,"(33.08, -96.61)"
2,75006,32.95,-96.89,,,,,"(32.95, -96.89)"
3,75007,33.00,-96.90,,,,,"(33.0, -96.9)"
4,75009,33.30,-96.80,,,,,"(33.3, -96.8)"


In [16]:
params = {
    "location": 
    "radius": 50,
    "types": "restaurant",
    "keyword": "ethnicity",
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through pandas dataframe
for index, row in restdata_df.iterrows():

    # get restaurant type from df
    restr_type = row['ethnicity']

    # add keyword to params dict
    params['keyword'] = restr_type

    # make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Highest Rated {restr_type} restaurant is {results[0]['name']}.")
        
        restdata_df.loc[index, 'Business'] = results[0]['name']
        restdata_df.loc[index, 'Cuisine'] = results[0]['ethnicity']
        restdata_df.loc[index, 'Price_Level'] = results[0]['price_level']
        restdata_df.loc[index, 'Rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

SyntaxError: invalid syntax (<ipython-input-16-cd007f3c2c2e>, line 7)